In [55]:
!pip install mediapipe==0.9.0.1 opencv-python==4.7.0.68 pandas scikit-learn==1.0.2 pyttsx3

In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

# Live View

In [12]:
cap = cv2.VideoCapture(1)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Live Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Capturing Landmarks & Exporting to CSV

In [3]:
import csv
import os
import numpy as np

In [13]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [10]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [11]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [15]:
class_name = "Heart Pain"

In [17]:
cap = cv2.VideoCapture(1)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow(str(class_name), image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Training a custom model

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [19]:
df = pd.read_csv('coords.csv')

In [20]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Normal,0.466250,0.331764,-0.907347,0.999906,0.488859,0.275133,-0.851194,0.999823,0.501955,...,0.001799,0.0,0.519976,0.273510,0.020471,0.0,0.524371,0.268968,0.021337,0.0
1,Normal,0.470933,0.328804,-0.778751,0.999902,0.490143,0.273135,-0.720188,0.999819,0.502533,...,0.002995,0.0,0.525391,0.273477,0.023576,0.0,0.529398,0.270235,0.024468,0.0
2,Normal,0.473655,0.327398,-0.766039,0.999898,0.490900,0.272356,-0.707355,0.999816,0.502857,...,0.002160,0.0,0.524868,0.273799,0.021655,0.0,0.528982,0.270436,0.022443,0.0
3,Normal,0.475893,0.325781,-0.764224,0.999896,0.492181,0.271684,-0.705913,0.999815,0.503827,...,0.003089,0.0,0.524431,0.274338,0.022729,0.0,0.528567,0.270888,0.023579,0.0
4,Normal,0.476316,0.324335,-0.698947,0.999903,0.492545,0.270901,-0.643586,0.999828,0.504012,...,0.003550,0.0,0.522296,0.274781,0.022839,0.0,0.526642,0.270979,0.023696,0.0


In [21]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
681,Heart Pain,0.603031,0.240310,-0.133693,0.999856,0.603485,0.217901,-0.105967,0.999742,0.604112,...,0.001849,0.0,0.615003,0.219073,0.018438,0.0,0.615705,0.217582,0.019649,0.0
682,Heart Pain,0.630482,0.239976,-0.098386,0.999840,0.630348,0.217015,-0.067377,0.999725,0.630086,...,0.002703,0.0,0.650602,0.225407,0.021623,0.0,0.652348,0.223076,0.023172,0.0
683,Heart Pain,0.660341,0.246722,-0.093042,0.999824,0.663124,0.226212,-0.062141,0.999707,0.662878,...,0.001548,0.0,0.680929,0.203546,0.016190,0.0,0.682565,0.200937,0.017109,0.0
684,Heart Pain,0.784059,0.248553,-0.158974,0.999406,0.790010,0.231295,-0.129459,0.999049,0.793062,...,0.000372,0.0,0.786331,0.217054,0.009484,0.0,0.788066,0.215969,0.009906,0.0
685,Heart Pain,0.763331,0.238170,-0.183521,0.999463,0.768428,0.220687,-0.152062,0.999140,0.771964,...,-0.000289,0.0,0.767391,0.206042,0.008283,0.0,0.768929,0.205090,0.008575,0.0


In [22]:
df[df['class']=='Normal']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Normal,0.466250,0.331764,-0.907347,0.999906,0.488859,0.275133,-0.851194,0.999823,0.501955,...,0.001799,0.0,0.519976,0.273510,0.020471,0.0,0.524371,0.268968,0.021337,0.0
1,Normal,0.470933,0.328804,-0.778751,0.999902,0.490143,0.273135,-0.720188,0.999819,0.502533,...,0.002995,0.0,0.525391,0.273477,0.023576,0.0,0.529398,0.270235,0.024468,0.0
2,Normal,0.473655,0.327398,-0.766039,0.999898,0.490900,0.272356,-0.707355,0.999816,0.502857,...,0.002160,0.0,0.524868,0.273799,0.021655,0.0,0.528982,0.270436,0.022443,0.0
3,Normal,0.475893,0.325781,-0.764224,0.999896,0.492181,0.271684,-0.705913,0.999815,0.503827,...,0.003089,0.0,0.524431,0.274338,0.022729,0.0,0.528567,0.270888,0.023579,0.0
4,Normal,0.476316,0.324335,-0.698947,0.999903,0.492545,0.270901,-0.643586,0.999828,0.504012,...,0.003550,0.0,0.522296,0.274781,0.022839,0.0,0.526642,0.270979,0.023696,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,Normal,0.461352,0.136900,-0.574556,0.999807,0.468961,0.111087,-0.548456,0.999576,0.474830,...,-0.000036,0.0,0.490685,0.114650,0.005235,0.0,0.492731,0.111651,0.005377,0.0
286,Normal,0.478395,0.116605,-0.568166,0.999825,0.487579,0.092118,-0.540122,0.999616,0.494293,...,0.000551,0.0,0.515690,0.089336,0.008127,0.0,0.517680,0.086542,0.008414,0.0
287,Normal,0.517985,0.096096,-0.530836,0.999840,0.528093,0.071382,-0.511509,0.999650,0.535316,...,0.000559,0.0,0.550757,0.064101,0.009792,0.0,0.552851,0.061340,0.010242,0.0
288,Normal,0.547569,0.083605,-0.527620,0.999854,0.554365,0.059128,-0.496080,0.999681,0.560227,...,0.001853,0.0,0.570215,0.046391,0.012356,0.0,0.572216,0.044333,0.012839,0.0


In [23]:
df[df['class']=='Heart Pain']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
290,Heart Pain,0.424330,0.284270,-0.781484,0.999926,0.440791,0.242940,-0.726478,0.999886,0.452241,...,0.004587,0.0,0.456013,0.238460,0.017930,0.0,0.459639,0.234844,0.018650,0.0
291,Heart Pain,0.424646,0.282075,-0.686610,0.999930,0.440889,0.240783,-0.638121,0.999887,0.452375,...,0.003736,0.0,0.457006,0.236550,0.017353,0.0,0.460560,0.233772,0.017991,0.0
292,Heart Pain,0.425431,0.279666,-0.679403,0.999932,0.441228,0.238658,-0.632693,0.999886,0.452743,...,0.003871,0.0,0.458188,0.236506,0.017996,0.0,0.461756,0.233614,0.018650,0.0
293,Heart Pain,0.425795,0.276422,-0.687483,0.999936,0.441324,0.236395,-0.639071,0.999887,0.452841,...,0.003659,0.0,0.458286,0.236288,0.017536,0.0,0.461841,0.233451,0.018162,0.0
294,Heart Pain,0.427420,0.273330,-0.657744,0.999939,0.442540,0.233989,-0.612027,0.999890,0.454376,...,0.003321,0.0,0.460228,0.233989,0.016921,0.0,0.463813,0.231225,0.017487,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,Heart Pain,0.603031,0.240310,-0.133693,0.999856,0.603485,0.217901,-0.105967,0.999742,0.604112,...,0.001849,0.0,0.615003,0.219073,0.018438,0.0,0.615705,0.217582,0.019649,0.0
682,Heart Pain,0.630482,0.239976,-0.098386,0.999840,0.630348,0.217015,-0.067377,0.999725,0.630086,...,0.002703,0.0,0.650602,0.225407,0.021623,0.0,0.652348,0.223076,0.023172,0.0
683,Heart Pain,0.660341,0.246722,-0.093042,0.999824,0.663124,0.226212,-0.062141,0.999707,0.662878,...,0.001548,0.0,0.680929,0.203546,0.016190,0.0,0.682565,0.200937,0.017109,0.0
684,Heart Pain,0.784059,0.248553,-0.158974,0.999406,0.790010,0.231295,-0.129459,0.999049,0.793062,...,0.000372,0.0,0.786331,0.217054,0.009484,0.0,0.788066,0.215969,0.009906,0.0


In [24]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [26]:
y_test

194        Normal
681    Heart Pain
659    Heart Pain
398    Heart Pain
290    Heart Pain
          ...    
34         Normal
91         Normal
410    Heart Pain
513    Heart Pain
111        Normal
Name: class, Length: 206, dtype: object

## Classification Model

In [4]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [5]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000)),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [33]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [34]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression(max_iter=1000))]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [35]:
fit_models['rc'].predict(X_test)

array(['Normal', 'Heart Pain', 'Heart Pain', 'Heart Pain', 'Normal',
       'Normal', 'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain',
       'Heart Pain', 'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain',
       'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain',
       'Normal', 'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain',
       'Normal', 'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain',
       'Heart Pain', 'Normal', 'Heart Pain', 'Normal', 'Normal',
       'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain', 'Heart Pain',
       'Normal', 'Normal', 'Heart Pain', 'Heart Pain', 'Heart Pain',
       'Normal', 'Normal', 'Heart Pain', 'Normal', 'Normal', 'Normal',
       'Heart Pain', 'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain',
       'Normal', 'Normal', 'Normal', 'Heart Pain', 'Heart Pain', 'Normal',
       'Normal', 'Normal', 'Normal', 'Heart Pain', 'Normal', 'Normal',
       'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain', 'Normal',
       'Hear

## Dump model

In [7]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [37]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9854368932038835
rc 0.9854368932038835
rf 1.0
gb 1.0


In [38]:
fit_models['rf'].predict(X_test)

array(['Normal', 'Heart Pain', 'Heart Pain', 'Heart Pain', 'Heart Pain',
       'Normal', 'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain',
       'Heart Pain', 'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain',
       'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain',
       'Normal', 'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain',
       'Normal', 'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain',
       'Heart Pain', 'Normal', 'Heart Pain', 'Normal', 'Normal',
       'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain', 'Heart Pain',
       'Normal', 'Normal', 'Heart Pain', 'Heart Pain', 'Heart Pain',
       'Normal', 'Normal', 'Heart Pain', 'Normal', 'Normal', 'Normal',
       'Heart Pain', 'Heart Pain', 'Heart Pain', 'Heart Pain',
       'Heart Pain', 'Normal', 'Normal', 'Normal', 'Heart Pain',
       'Heart Pain', 'Normal', 'Normal', 'Normal', 'Normal', 'Heart Pain',
       'Normal', 'Normal', 'Heart Pain', 'Normal', 'Heart Pain',
       'Heart Pain', 'Normal

In [39]:
y_test

194        Normal
681    Heart Pain
659    Heart Pain
398    Heart Pain
290    Heart Pain
          ...    
34         Normal
91         Normal
410    Heart Pain
513    Heart Pain
111        Normal
Name: class, Length: 206, dtype: object

In [40]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# Detection

In [16]:
import pyttsx3
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [17]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [18]:
# feature names
fnames = []
for val in range(1, num_coords+1):
    fnames += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [19]:
engine = pyttsx3.init()
cap = cv2.VideoCapture(1)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Make Detections
            X = pd.DataFrame([row], columns=fnames)
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            #print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Audio Alert
            if body_language_class != "Normal":
                engine.say(str(body_language_class)+"detected")
                engine.runAndWait()
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()